In [ ]:
from notebook_prelude import *

In [ ]:
from utils import cooccurrence

text = 'Quotation is a serviceable substitute for wit'
# https://en.wikiquote.org/wiki/Jorge_Luis_Borges
text = 'Universal history is the history of a few metaphors'.lower()
graphs = []
for window_size in [1, 2, 6]:
    mat = cooccurrence.get_coocurrence_matrix(text, window_size=window_size)
    g = graph_helper.convert_from_numpy_to_nx(*mat)
    graphs.append((window_size, g))

In [ ]:
FOLDER = 'tmp/cooccurrence_graphs_example'
os.makedirs(FOLDER, exist_ok=True)
for window_size, graph in graphs:
    nx.write_dot(graph, '{}/window_size_{}.dot'.format(FOLDER, window_size))

In [ ]:
NUM_ITEMS = len(set(text.split()))
ANGLES = 2 * np.pi / NUM_ITEMS
FACTOR = 120
xys = []
for i in range(NUM_ITEMS):
    angle_in_rad = ANGLES * i
    x, y = np.sin(angle_in_rad), np.cos(angle_in_rad)
    xys.append((x, y))
xs = [x for x, _ in xys]
ys = [y for _, y in xys]
max_x = max(xs)
min_x = min(xs)
max_y = max(ys)
min_y = min(ys)
xys_ = []
for x, y in xys:
    x = (x - min_x) * FACTOR
    y = (y - min_y) * FACTOR
    xys_.append((x, y))
print(xys_)
pos = ['universal', 'history', 'is', 'the', 'of', 'a', 'few', 'metaphors']
for item, (x, y) in zip(pos, xys_):
    for file in glob('{}/*.dot'.format(FOLDER)):
        with open(file) as f:
            graph = f.read()
        graph_ = graph.replace('{};'.format(item), '{}[pos = "{:.2f},{:.2f}!"];'.format(item, x, y))
        with open(file, 'w') as f:
            f.write(graph_)

In [ ]:
!cd tmp/cooccurrence_graphs_example && ./convert.sh

In [ ]:
g = graphs[0][1]
print(nx.adjacency_matrix(g).todense())
for layout in ['circular_layout', 'fruchterman_reingold_layout', 'spectral_layout', 'spring_layout', 'shell_layout']:
    l = getattr(nx.layout, layout)
    pos = l(g)
    fig, axes = plt.subplots(ncols=len(graphs))
    for ax, (window_size, graph) in zip(axes, graphs):
        nx.draw_networkx(graph, pos = pos, ax=ax)
        ax.grid('off')
        cleanup_axes(ax)
        ax.set_title(window_size)
    fig.suptitle(layout)